In [1]:
from models import NERModel, SamModelPrediction
import json
import cv2
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import numpy as np
import torch
import os


/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/Project/vqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
flan = NERModel("./fineTunedT5ForNER")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [44]:
def extract_entities(sentence):
    instruction = f" Extract exact words from the sentence that are either common nouns or proper noun from the sentence"
    
    prompt = f"""
    1. Sentence: "The cat sat on the mat?" Common Nouns: cat, sat, mat.
    2. Sentence: "He drinks a lot of coffee and reads many books." Common Nouns:  coffee, reads, books.
    3. Sentence: "Sky is blue and grass is green." Common Nouns: sky, blue, grass, green.
    4. Sentence: "Is the  Boy is skateboarding on the wall?" Common Nouns: boy, skateboarding, wall .
    5. Sentence: "What color is teddy bear " Common Nouns: teddy bear, color
    6. Sentence: "Teddy bear is pink ?" Common Nouns: teddy bear, pink
    Sentence: "{sentence}"
    Common Nouns:"""
    # Encode the prompt
    
    
    # Generate output
    outputs = flan.extract_entities(instruction,prompt)
    entity_list = [entity.strip() for entity in outputs.split(',')]
    # Decode and process the output
  
   
    return entity_list

In [46]:
extract_entities("Boys doing skateboarding behind")

hello


'Boys, skateboarding'

In [3]:
file_path = '/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/Project/dataset/combined_data.json'
ner = NERModel()

sam_predictor = SamModelPrediction()

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


final text_encoder_type: bert-base-uncased
Model loaded from /Users/sudhanshu/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [17]:
x=ner.extract_entities("Is TeddyBear pink?")

In [41]:
print(x)
entities= getEntities(x)
print(entities)

[{'entity': 'I-PER', 'score': 0.4765262, 'index': 2, 'word': 'Teddy', 'start': 3, 'end': 8}, {'entity': 'I-ORG', 'score': 0.8116944, 'index': 3, 'word': '##B', 'start': 8, 'end': 9}, {'entity': 'I-ORG', 'score': 0.7172411, 'index': 4, 'word': '##ear', 'start': 9, 'end': 12}]
['Teddy', '##B', '##ear']


Testing SAM

In [12]:
x= ner.extract_entities("Apples are born in america")
print(x)


[{'entity': 'I-LOC', 'score': 0.6480648, 'index': 7, 'word': '##eric', 'start': 21, 'end': 25}]


In [35]:
image_path = "/Users/sudhanshu/Desktop/UMASS_COURSES_SEMESTERS/SEM_2/NLP/Project/testSam.jpeg"
image=Image.open(image_path).convert("RGB")

# Use the predict_image method with the loaded image and a text prompt
entities=["hello","a","0"]
masks = [sam_predictor.predict_image(image, ent) for ent in entities]


In [53]:

def show_mask(mask, ax, random_color=False):
    """
    Adds a mask to an existing Axes object with the option to use a random color.

    Args:
    mask (array): A 2D numpy array representing the mask.
    ax (matplotlib.axes.Axes): The Axes object where the mask will be added.
    random_color (bool): If True, the mask will be displayed in a random color.
    """
    if random_color:
        # Generate a random color with transparency
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        # Use a specific color with transparency
        color = np.array([30/255, 144/255, 255/255, 0.6])

    h, w = mask.shape
    # Multiply the mask by the color and reshape for proper display
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image, interpolation='none', alpha=0.5)



In [46]:
def create_segmented_image(pil_image, masks, background_color=(0, 0, 0, 0)):
    """
    Create a segmented image where only the areas defined by masks are visible,
    and the rest is set to a specified background color.

    Args:
    pil_image (PIL.Image): The original image.
    masks (list of numpy arrays): A list of 2D numpy arrays representing masks.
    background_color (tuple): RGBA color tuple for the background, defaults to transparent.

    Returns:
    PIL.Image: The segmented image with only masked areas from the original image.
    """
    if  len(masks) ==0:  # Check if the masks list is empty
        return pil_image
    # Convert the PIL Image to a NumPy array (RGBA for transparency handling)
    image_np = np.array(pil_image.convert('RGB'))

    # Prepare a blank canvas for the final image with the background color
    final_image_np = np.full(image_np.shape, background_color, dtype=np.uint8)

    # Process each mask
    for mask in masks:
        # Apply the mask to copy relevant parts from the original image to the final image
        final_image_np[mask] = image_np[mask]

    # Convert back to PIL Image
    segmented_image = Image.fromarray(final_image_np, 'RGB')
    return segmented_image

In [ ]:

#mask=mask.numpy()
# mask = torch.from_numpy(mask)
print(type(masks))

#masks = [item for sublist in masks for item in sublist]
result_image = create_segmented_image(image, masks)
directory = "segmentedImage"
print(result_image.mode)
filename="hola"
result_image.save(os.path.join(directory, filename))

NER and SAM applied on Dataset

flanT5 = 